In [15]:
import pandas as pd, numpy as np, glob
from pathlib import Path
import re

def natural_key(p):                     # сортировка raw1, raw2, …
    parts = re.split(r"(\d+)", p.stem)
    return [int(s) if s.isdigit() else s for s in parts]

all_rows = []
DATA_DIR = Path("../full_dataset/circle")
PATTERN   = "*.csv" 


for fn in sorted(DATA_DIR.glob(PATTERN), key=natural_key):
    df = pd.read_csv(fn)
    # scale steering to [-1,1]
    STEER_MIN, STEER_MAX = 1968, 4004
    STEER_C   = (STEER_MIN + STEER_MAX) / 2          # 2986
    STEER_SPAN= (STEER_MAX - STEER_MIN) / 2          # 1018
    
    df["a"] = (df["steer"] - STEER_C) / STEER_SPAN   # →  [-1 … +1]
    
    # next-step columns
    df["yawRate_n"]     = df["yawRate"].shift(-1)
    df["ay_world_n"]    = df["ay_world"].shift(-1)
    df["ax_n"]          = df["ax"].shift(-1)
    df["angular_accel_n"] = df["angular_accel"].shift(-1)
    df = df.dropna()
    print(df)

    s  = df[["yawRate","ay_world","ax","angular_accel"]].values
    a  = df[["a"]].values
    sn = df[["yawRate_n","ay_world_n","ax_n","angular_accel_n"]].values
    r  = -np.abs(df["yawRate"]) - 0.5*np.abs(df["ay_world"])
    all_rows.append(np.hstack([s, a, sn, r.values[:,None]]))

np.savez("old_staff/model_dataset.npz", data=np.vstack(all_rows))

       t      ax      ay   yawRate  steer  t_sec         yaw    yaw_rad  \
0   1202  0.1118  0.2270    0.2134   2820  1.202    0.034571   0.000603   
1   1255  0.0631  0.3109    1.7378   2970  1.255    0.126674   0.002211   
2   1309  0.1261  0.3698    0.7317   2948  1.309    0.166186   0.002900   
3   1363  0.3102  0.5118   -0.0610   2168  1.363    0.162892   0.002843   
4   1418  0.2596  0.5095    1.2195   1968  1.418    0.229964   0.004014   
..   ...     ...     ...       ...    ...    ...         ...        ...   
69  4927  2.8672  2.8398 -295.2439   1968  4.927 -589.753361 -10.293138   
70  4980  2.3273  2.4586 -297.4695   1966  4.980 -605.519244 -10.568304   
71  5035  2.2454  2.3054 -300.8842   1968  5.035 -622.067875 -10.857133   
72  5089  1.4640  2.4443 -295.3964   1968  5.089 -638.019281 -11.135537   
73  5143  1.2038  2.6919 -291.0976   1968  5.143 -653.738551 -11.409890   

    ax_world  ay_world  angular_accel  is_drift         a  yawRate_n  \
0   0.111663  0.227067     

In [16]:
import torch, torch.nn as nn, torch.optim as optim, numpy as np

data = np.load("old_staff/model_dataset.npz")["data"]
S, A, Sn = data[:,:4], data[:,4:5], data[:,5:9]

# нормализация
mu_s,  sig_s  = S.mean(0),  S.std(0)+1e-6
mu_a,  sig_a  = A.mean(0),  A.std(0)+1e-6
mu_sn, sig_sn = Sn.mean(0), Sn.std(0)+1e-6

def norm(x, mu, sig): return (x - mu) / sig

# --- ВАЖНО: сразу .astype(np.float32) или потом .float() ---
S  = torch.from_numpy(norm(S , mu_s , sig_s )).float()
A  = torch.from_numpy(norm(A , mu_a , sig_a )).float()
Sn = torch.from_numpy(norm(Sn, mu_sn, sig_sn)).float()

ds     = torch.utils.data.TensorDataset(torch.cat([S, A], 1), Sn)
loader = torch.utils.data.DataLoader(ds, batch_size=1024, shuffle=True)

net = nn.Sequential(
        nn.Linear(5, 128), nn.ReLU(),
        nn.Linear(128, 128), nn.ReLU(),
        nn.Linear(128, 4)          # предсказываем нормализ. s_{t+1}
)

opt = optim.Adam(net.parameters(), 1e-3)

for epoch in range(200):
    for x, y in loader:
        out  = net(x)
        loss = ((out - y) ** 2).mean()
        opt.zero_grad()
        loss.backward()
        opt.step()

In [17]:
torch.save({"net":net.state_dict(),
            "mu_s":mu_s, "sig_s":sig_s,
            "mu_a":mu_a, "sig_a":sig_a,
            "mu_sn":mu_sn,"sig_sn":sig_sn},
           "old_staff/dyn_model.pt")